In [97]:
import re
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [98]:
raw_html = urlopen("http://www.dreambank.net/random_sample.cgi?min=0&max=99999&n=99999").read()
soup_doc = BeautifulSoup(raw_html, "html.parser")

In [99]:
dreamers = []

for o in soup_doc.find_all('option'):
    
    
    if len(o.text.split(':')) > 1:
        desc = o.text.split(':')[1][1:].split('[n=')[0].replace(']', '')
    else:
        desc = "NaN"
    
    name = o.text.split(':')[0].split('[n=')[0]
    number_of_dreams = int(o.text.split('[n=')[1].replace(']', ''))
    
    dreamers.append({
        'name': name,
        'description': desc,
        'id': o['value'],
        'number_of_dreams': number_of_dreams
    })

dreamers = pd.DataFrame(dreamers)
dreamers.head()

,description,id,name,number_of_dreams
0,a detailed dreamer,alta,Alta,422
1,age 18 & 20,angie,Angie,48
2,a middle-aged woman,arlie,Arlie,212
3,NaN,b,Barb Sanders,3116
4,NaN,b2,Barb Sanders #2,1138


In [100]:
german_dreamers = [
    'german-f.de',
    'german-m.de',
    'zurich-f.de',
    'zurich-m.de',
    'vonuslar.de'
]

dreamers = dreamers[~dreamers['id'].isin(german_dreamers)]

In [101]:
dreams = []

for index, dreamer in dreamers.iterrows():
    raw_html = urlopen("http://www.dreambank.net/random_sample.cgi?series="
                       +dreamer['id']+"&min=0&max=9000&n=99999").read()
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    
    for d in soup_doc.find_all('span'):
        dream = {
            'content': d.text
        }
        dream.update(dreamer)
        dreams.append(dream)
        
dreams = pd.DataFrame(dreams)
dreams.head()

,content,description,id,name,number_of_dreams
0,"\n#1 (1957)The one at the Meads's house, where...",a detailed dreamer,alta,Alta,422
1,\n#2 (8/11/67)I'm at a family reunion in a lar...,a detailed dreamer,alta,Alta,422
2,\n#3 (8/1/85)I watch a plane fly past and shor...,a detailed dreamer,alta,Alta,422
3,\n#4 (1985?)Me pulling the green leaves and be...,a detailed dreamer,alta,Alta,422
4,\n#5 (1985?)I'm in a room that reminds me of (...,a detailed dreamer,alta,Alta,422


In [102]:
dreams.drop(['number_of_dreams', 'description', 'name'], axis=1, inplace=True)

In [103]:
dreams.head()

,content,id
0,"\n#1 (1957)The one at the Meads's house, where...",alta
1,\n#2 (8/11/67)I'm at a family reunion in a lar...,alta
2,\n#3 (8/1/85)I watch a plane fly past and shor...,alta
3,\n#4 (1985?)Me pulling the green leaves and be...,alta
4,\n#5 (1985?)I'm in a room that reminds me of (...,alta


In [105]:
dreams.to_csv('dreams.csv', index=False)